# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [594]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [595]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [735]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

#Исключение для случая, когда производной в точке не существует
class NoDerivativeException(Exception):
    pass

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  
    __radd__ = __add__
    
    # Унарные операции
    def __neg__(self) -> "Dual":
        return   Dual(-1 * self.value, -1 * self.d)
    
    def __pos__(self) -> "Dual":
        return Dual(self.value, self.d)
    
    def __abs__(self) -> "Dual":
        if self.value < 0:
            return Dual(-1 * self.value, -1 * self.d)
        elif self.value > 0:
            return Dual(self.value, self.d)
        else:
            raise NoDerivativeException
        
    #Возведение в степень    
    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Number():
                if other == 0:
                    return Dual(1, 0)
                return Dual(self.value ** float(other), float(other) * self.d * (self.value ** (float(other) - 1)))
    
    #Деление
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Number():
                return Dual(self.value / float(other), self.d / float(other))
            case Dual(o_value, o_d):
                if o_value == 0:
                    raise NoDerivativeException
                return Dual(self.value / o_value, (self.d * o_value - o_d * self.value) / (o_value) ** 2)

    #Деление числа на экземпляр класса  
    def __rtruediv__(self, other: Number) -> "Dual":
        if self.value == 0:
            raise NoDerivativeException
        return float(other) * (self ** (-1))
    
    # Вычитание
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)
    
    #Вычитание справа
    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value - self.value, o_d - self.d)
            case Number():
                return Dual(float(other) - self.value, -1 * self.d)
            

    
def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    def Der(x: float) -> float:
        try:
            res = func(Dual(x, 1.0))
            match res:
                case Number():
                    res = 0
                case Dual(o_value, o_d):
                    res = func(Dual(x, 1.0)).d
        except NoDerivativeException:
            print("Function has no Derivative in this point")
        else:
            return res
    return Der

Тесты:

In [736]:
eps: float = 10e-5
f1 = lambda x: -6 * x
f1_diff = diff(f1)
f1_diff(3)
assert(abs(f1_diff(3) - (-6)) < eps)


f2 = lambda x: abs(x ** 2)
f2_diff = diff(f2)
assert(abs(f2_diff(-1) - (-2)) < eps)

f3 = lambda x: x / 12
f3_diff = diff(f3)
assert(abs(f3_diff(4) - (1 / 12)) < eps)

f4 = lambda x: x / x ** 2 + 5 * x ** 6 + x ** 3 + 2
f4_diff = diff(f4)
assert(abs(f4_diff(3) - (65852 / 9)) < eps)

f5 = lambda x:  x * 1 / 5 - x ** 0 + 1
f5_diff = diff(f5)
assert(abs(f5_diff(0) - (0.2)) < eps)

f6 = lambda x: 1 / abs(x)
f6_diff = diff(f6)
assert(abs(f6_diff(1) - (-1)) < eps)

Тест для случая, когда производная не существует

In [737]:
f6 = lambda x: 1/abs(x)
f6_diff = diff(f6)
f6_diff(0)

Function has no Derivative in this point


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [738]:
import math
#Экспонента
def exp(val: Union["Dual", Number]) -> Union["Dual", Number]:
    match val:
        case Number():
            return math.exp(val)
        case Dual(value, d):
            return Dual(math.exp(value),d * math.exp(value))
        
#синус
def sin(val: Union["Dual", Number]) -> Union["Dual", Number]:
    match val:
        case Number():
            return math.sin(val)
        case Dual(value, d):
            return Dual(math.sin(value), d * math.cos(value))
        
#косинус       
def cos(val: Union["Dual", Number]) -> Union["Dual", Number]:
    match val:
        case Number():
            return math.cos(val)
        case Dual(value, d):
            return Dual(math.cos(value), -1 * d * math.sin(value))
        
# натуральный логорифм      
def log(val: Union["Dual", Number]) -> Union["Dual", Number]:
    match val:
        case Number():
            if val <= 0:
                raise NoDerivativeException
            return math.log(val)
        case Dual(value, d):
            if value <= 0:
                raise NoDerivativeException
            return Dual(math.log(value), d / value)

Тесты

In [715]:
eps: float = 10e-5
f1 = lambda x: sin(1 / (x ** 2))
f1_diff = diff(f1)
assert(abs(f1_diff(math.pi) - (-0.0641723)) < eps)

f2 = lambda x: cos(x ** 3 + 2 * x + 3)
f2_diff = diff(f2)
assert(abs(f2_diff(1) - (1.39708)) < eps)

f3 = lambda x: exp(5 * x **3 + cos(2 * x) + 1 / x)
f3_diff = diff(f3)
assert(abs(f3_diff(0.5) - (-45.8028)) < eps)

f4 = lambda x: log (x / x ** 2 + 5 * x ** 6 + x ** 3 + 2)
f4_diff = diff(f4)
assert(abs(f4_diff(2) - (2.94024)) < eps)

f5 = lambda x:  log( exp(x ** 2 + cos(37 * x + 1)) - 26 * sin(-8 * log(x) / (x ** 3)))
f5_diff = diff(f5)
assert(abs(f5_diff(12) - (57.0774)) < eps)

f6 = lambda x:  exp(-x ** 2)
f6_diff = diff(f6)
assert(abs(f6_diff(0.5) - (-0.778801)) < eps)


In [716]:
f7 = lambda x:  log(x)
f7_diff = diff(f5)
f7_diff(0)

Function has no Derivtive in this point


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [717]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

C:\Users\User\AppData\Local\Temp\ipykernel_1584\2371880734.py:6: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  derivative(f, 2.)


22.0

Функция из scipy в некорорых случаях считает неверно

Функция для численного дифференцирования с точностью $h^{2}$

In [718]:
def numDer(f: Callable[[float], float], x: float, h: float) -> float:
    try: 
        v: float = f(x) #проверка имеет ли функция значение в точке дифференциирования
        res: float = (f(x + h) - f(x - h)) / (2 * h)
    except ValueError:
        print('Function has no Derivtive in this point')
    else:
        return res

Случай когда производной нет

In [719]:
f1 = lambda x: math.log(x)
numDer(f1, 0, eps)

Function has no Derivtive in this point


In [720]:
eps: float = 10e-5
f1 = lambda x: sin(1 / (x ** 2))
f1_diff = diff(f1)
assert(abs(f1_diff(math.pi) - numDer(f1, math.pi, eps)) < eps)

f2 = lambda x: cos(x ** 3 + 2 * x + 3)
f2_diff = diff(f2)
assert(abs(f2_diff(1) - numDer(f2, 1, eps)) < eps)

f3 = lambda x:  log( exp(x ** 2 + cos(37 * x + 1)) - 26 * sin(-8 * log(x) / (x ** 3)))
f3_diff = diff(f3)
assert(abs(f3_diff(12) - numDer(f3, 12, eps)) < eps)

f4 = lambda x: abs(x ** 2)
f4_diff = diff(f4)
assert(abs(f4_diff(-1) - numDer(f4, -1, eps)) < eps)

f5 = lambda x: x / 12
f5_diff = diff(f5)
assert(abs(f5_diff(4) - numDer(f5, 4, eps)) < eps)

f6 = lambda x: x / x ** 2 + 5 * x ** 6 + x ** 3 + 2
f6_diff = diff(f6)
assert(abs(f6_diff(3) - numDer(f6, 3, eps)) < eps)

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [721]:
import random
from typing import List

#создание одного выражения
def createExample(max_elems: int = 4, max_depth : int = 3) -> Callable[[float], float]:
    #глубина сложных функций
    depth_of_complex: int = random.randint(0, max_depth)
    bin_ops = [' + ', ' - ',' / ',' * ']
    un_ops = ['abs', ' ** ', 'log', 'sin', 'cos', 'exp', 'coef']
    #Число выражений, участвующих в бинарных операциях
    n: int = random.randint(1, max_elems)
    #вырожденый случай f = x / const
    if depth_of_complex == 0:
        x = random.choices(['x', 'num'], weights=[0.95, 0.05])[0]
        if x == 'num':
            x = random.uniform(-15, 15)
            if x < 0:
                x = '(' + str(x) + ')'
            else:
                x = str(x)
        return x
    #невырожденный случай
    bins: List = []
    for i in range(n - 1):
        bins.append(random.choices(bin_ops, weights=[0.4, 0.4, 0.1, 0.1])) #взвешенный выбор
    ans: List = []
    elems: List = []
    for i in range(n):
        eq: str = createExample(max_elems, max_depth - 1)
        un:str = random.choices(un_ops)[0]
        match un:
            case 'coef':
                k: int = random.randint(-7, 7)
                if k < 0:
                    k = '(' + str(k) + ')'
                elems.append(str(k) + ' * ' + eq)
            case ' ** ':
                if ' ** ' in eq:
                    k: int = random.randint(1, 6)
                else:
                    k: int = random.randint(-6, 6)
                if k < 0:
                    k: int = '(' + str(k) + ')'
                elems.append(eq + ' ** ' + str(k))
            case 'log':
                gtz = random.choice([' ** ', 'abs'])
                match gtz: #упрощение для того чтобы подлогарифмическое не давало сложных областей определения
                    case ' ** ':
                        if ' ** ' in eq:
                            k: int = random.randrange(2, 6, step = 2)
                        else:
                            k: int = random.randrange(-6, 6, step = 2)
                        if k < 0:
                            k: int = '(' + str(k) + ')'
                        eq = '(' + eq + ')' + ' ** ' + str(k)
                    case _:
                        curr_f = eval('lambda x: ' + eq)
                        if curr_f(13.345) == 0 and curr_f(-13.345):
                            eq = '0'
                        else:
                            eq = gtz + '(' + eq + ')'
                elems.append('log' + '(' + eq + ' + 1)')
            case 'abs':
                curr_f = eval('lambda x: ' + eq)
                if curr_f(13.345) == 0 and curr_f(-13.345): #упрощение под знаком модуля
                    elems.append('0')
                else:
                    elems.append(un + '(' + eq + ')')
            case _:
                elems.append(un + '(' + eq + ')')
    ans.append(elems[0])
    for i in range(1, n):
        ans.append(bins[i - 1][0])
        ans.append(elems[i])
    ans = ''.join(ans)
    return ans

#Программа для тестировки нескольких вариантов
def test(f_sym: Callable[[float], float], f_num: Callable[[float], float], n_tests: int = 100, eps: float = 10e-3) -> None:
    n_errs: int = 0
    problem_funs: List = []
    for i in range(n_tests):
        while True:
            try:
                eq = createExample()
                x = random.uniform(-3, 3)
                fun = eval('lambda x: ' + eq)
                fun(x)
                f_sym_diff = f_sym(fun)
                #print(eq, x) вывод промежуточных функций для дебага
                while abs(f_sym_diff(x)) > 10e5:
                    eq = createExample()
                    x = random.uniform(-3, 3)
                    fun = eval('lambda x: ' + eq)
                    f_sym_diff = f_sym(fun)
                    #print(eq, x) вывод промежуточных функций для дебага
                value = f_sym_diff(x) - f_num(fun, x, eps ** 3)
            except ZeroDivisionError:
                pass
            except NoDerivativeException:
                pass
            except OverflowError:
                pass
            else:
                break
        if abs(value) >= eps:
            n_errs += 1
            problem_funs.append((value, f_sym_diff(x), f_num(fun, x, eps ** 3), eq, x))
    print(f'Symbolic derivative and numerical derivative are equal for {n_tests - n_errs}')
    print(f'Errors in {n_errs} cases')
    if n_errs > 0:
        print('Problem funcs:')
        for elem in problem_funs:
            print(elem)


Тест функции создания примера

In [726]:
createExample()

'(-7) * x - exp(log(0 + 1) - 1 * exp(x) - 7 * x)'

Сравнение численного и символьного дифференциирования. Проблема может возникать из-за большого аболютного значения второй производной, что напрямую влияет на точность вычисления производной по численной формуле

In [723]:
test(diff, numDer)

Symbolic derivative and numerical derivative are equal for 98
Errors in 2 cases
Problem funcs:
(59989.12035818864, 437779.77124293294, 377790.6508847443, 'log(0 + 1) - 7 * log((x) ** (-2) + 1) + cos(x ** (-3) - (-4) * x - exp(x)) ** 3 * cos(x)', -0.0530052433728887)
(0.45520560457592296, -167.18712839509266, -167.64233399966858, '0 - exp(log((1 * x - exp(x) * sin(x)) ** 0 + 1) + cos(x ** (-4) + exp(x) - 4 * x) - 7 * exp(x) + 0 * (-2) * x) + log(0 + 1)', -0.13136705191194054)


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [727]:
def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    def Der(x: float, y: float, z: float) -> float:
        ans: List = []
        vars: List = [x, y, z]
        for i in range(len(vars)):
            curr = vars.copy()
            curr[i] = Dual(vars[i], 1.0)
            x_new = curr[0]
            y_new = curr[1]
            z_new = curr[2]
            try:
                res = func(x_new, y_new, z_new)
                match res:
                    case Number():
                        res = 0
                    case Dual(o_value, o_d):
                        res = res.d
                ans.append(res)
            except NoDerivativeException:
                print("Function has no Derivtive in this point")
                return
        else:
            return ans
    return Der

In [728]:
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]

[10.0, 5.0, 1.0]

In [730]:
def f(x: float, y: float, z: float) -> float:
    return abs(x) + y ** 2 - z 


f_diff = diff(f)

f_diff(0, 2, 3) # No Derivative

Function has no Derivtive in this point


In [734]:
def f(x: float, y: float, z: float) -> float:
    return log(x * y * z) + x * sin(z) - 1 / z


f_diff = diff(f)

print(f_diff(1, 1, 1), [sin(1) + 1, 1/1, 1 * cos(1) + 1 + 1 /1 ** 2])

[1.8414709848078965, 1.0, 2.5403023058681398] [1.8414709848078965, 1.0, 2.5403023058681398]
